# Load Models

In the previous notebook we saved the large model. In this notebook we have a look on how to reload it.

## Setup & Data

In [14]:
# Import packages
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

tf.keras.backend.set_floatx('float64')

# Define random seed for whole notebook
RSEED=42

In [15]:
# Load data
df = pd.read_csv("../data/boston.csv")

# Define target 
y = df.pop("MEDV")

# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(df, y, random_state=RSEED)

In [16]:
# Scale numerical features
# Scale numerical values
col_scale = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'LSTAT']

scaler = StandardScaler()
X_train[col_scale] = scaler.fit_transform(X_train[col_scale])
X_test[col_scale] = scaler.transform(X_test[col_scale])

# Convert to np array
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [17]:
ls saved_model

 Volume in drive D is Vault-SSD
 Volume Serial Number is 7CCF-4F05

 Directory of d:\Programming\ai_ds_bootcamp\ds-artificial-neural-networks\day_2\saved_model

06/01/2026  14:52    <DIR>          .
06/01/2026  14:21    <DIR>          ..
06/01/2026  14:44    <DIR>          my_large_model
06/01/2026  14:52    <DIR>          my_large_model_2
               0 File(s)              0 bytes
               4 Dir(s)  727,376,703,488 bytes free


In [18]:
ls saved_model/my_large_model

Invalid switch - "my_large_model".


## SavedModel format

Reload a fresh Keras model from the saved model:

In [19]:
# Load the saved model
with tf.device('/cpu:0'):
    new_large_model = tf.keras.models.load_model('saved_model/my_large_model_2')

# Check its architecture
new_large_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               6656      
                                                                 
 dense_8 (Dense)             (None, 512)               262656    
                                                                 
 dense_9 (Dense)             (None, 512)               262656    
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dense_11 (Dense)            (None, 1)                 513       
                                                                 
Total params: 795137 (6.07 MB)
Trainable params: 795137 (6.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [21]:
# Evaluate the restored model
with tf.device('/cpu:0'):
    loss, mse = new_large_model.evaluate(X_test, y_test, verbose=2)
print(f'Model MSE: {mse}')

4/4 - 0s - loss: 2.1907 - mse: 11.6384 - 29ms/epoch - 7ms/step
Model MSE: 11.638398409902056


The Model MSE is higher than in the notebook before, even though we are using the same data-split. We are using the same model and the same data. Therefore, we would assume that we also receive the same MSE. 

> **Exercise:** Can you find out where this notebook varies from the procedure in the notebook before? It might help to have a look at the values in X_train in both notebooks.

<details><summary>
Click here for a hint...
</summary>
Check out how the data were preprocessed. Did both notebooks use the same scaler?
</details>